In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD

In [13]:
#ModelCreate#Applies to GS1, GS2, GS4, GS5
def create_model(optimizer='adam', init_mode='uniform', activation='relu', dropout_rate=-1, weight_constraint=-1, neurons=16):
	# create model
	model = Sequential()
	if dropout_rate > -1:#This is for Case 6.
	    model.add(Dense(neurons, input_dim=13, activation=activation, kernel_initializer=init_mode, kernel_constraint=maxnorm(weight_constraint)))
	    model.add(Dropout(dropout_rate))
	else:
	    model.add(Dense(neurons, input_dim=13, kernel_initializer=init_mode, activation=activation))        
	model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model

In [8]:
##GS3
def create_model3(learn_rate=0.01, momentum=0):
	# create model
	model = Sequential()
	model.add(Dense(16, input_dim=13, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	optimizer = SGD(lr=learn_rate, momentum=momentum)
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model

In [5]:
def gs1Spec():
    model = KerasClassifier(build_fn=create_model, verbose=0)
    # define the grid search parameters
    batch_size = [10, 20, 40, 60, 80, 100]
    epochs = [10, 50, 100]
    param_grid = dict(batch_size=batch_size, epochs=epochs)
    return model, batch_size, epochs, param_grid

In [ ]:
def gs2Spec(batch_size=10, epochs=100):
    #####GS2
    # create model
    model = KerasClassifier(build_fn=create_model, verbose=0, batch_size=batch_size, epochs=epochs)

    # define the grid search parameters
    optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
    param_grid = dict(optimizer=optimizer)
    
    return model, optimizer, param_grid

In [7]:
#GS3 is solely for testing SGD, and affects no other tests.
def gs3Spec(batch_size=10, epochs=100):
    # create model
    model = KerasClassifier(build_fn=create_model3, verbose=0, batch_size=batch_size, epochs=epochs)

    # define the grid search parameters
    learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
    momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
    param_grid = dict(learn_rate=learn_rate, momentum=momentum)
    return model, learn_rate, momentum, param_grid

In [10]:
def gs4Spec(batch_size=10, epochs=100, optimizer='adam'):
    # create model
    model = KerasClassifier(build_fn=create_model(optimizer), verbose=0, batch_size=batch_size, epochs=epochs)
    init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
    param_grid = dict(init_mode=init_mode)
    return model, init_mode, param_grid

In [ ]:
def gs5Spec(batch_size=10, epochs=100, optimizer='adam', init_mode='uniform'):
    # create model
    model = KerasClassifier(build_fn=create_model(optimizer, init_mode), verbose=0, batch_size=batch_size, epochs=epochs)
    activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
    param_grid = dict(activation=activation)
    
    return model, activation, param_grid

In [14]:
def gs6Spec(batch_size=10, epochs=100, optimizer='adam', init_mode='uniform', activation='relu'):
    # create model
    model = KerasClassifier(build_fn=create_model(optimizer, init_mode, activation, dropout_rate=0.0, weight_constraint=0), verbose=0, batch_size=batch_size, epochs=epochs)
    weight_constraint = [1, 2, 3, 4, 5]
    dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
    
    return model, weight_constraint, dropout_rate, param_grid

In [ ]:
def gs7Spec(batch_size=10, epochs=100, optimizer='adam', init_mode='uniform', activation='relu', dropout_rate=0.0, weight_constraint=0):
    # create model
    model = KerasClassifier(build_fn=create_model(optimizer, init_mode, activation, dropout_rate=0.0, weight_constraint=0, dropout_rate=dropout_rate, weight_constraint=weight_constraint), verbose=0, batch_size=batch_size, epochs=epochs)
    neurons = [1, 5, 10, 15, 20, 25, 30]
    param_grid = dict(neurons=neurons)
    return model, neurons, param_grid

In [ ]:
def studentSetup():
    ####Education Data Setup
    # load dataset
    dataset = pd.read_csv("slim-xAPI-Edu-Data.csv")
    #students failed if they are in class L.
    dataset['Failed'] = np.where(dataset['Class'] == 'L', True, False)
    dataset['gender'] = np.where(dataset['gender']=='M',1,0)
    dataset['Relation'] = np.where(dataset['Relation']=='Father',1,0)
    dataset['ParentAnsweringSurvey'] = np.where(dataset['ParentAnsweringSurvey'] == 'Yes', 1, 0)
    dataset['ParentschoolSatisfaction'] = np.where(dataset['ParentschoolSatisfaction'] == 'Yes', 1, 0)
    dataset['AbsentMoreThanWeek'] = np.where(dataset['StudentAbsenceDays'] == 'Above-7', 1, 0)
    dataset['Semester'] = np.where(dataset['Semester'] == 'F', 1, 0)

    X = dataset[['raisedhands', 'VisITedResources', 'SectionID', 'Topic', 'StageID', 'AnnouncementsView', 'Semester', 'Discussion', 'gender', 'Relation', 'ParentAnsweringSurvey', 'ParentschoolSatisfaction', 'AbsentMoreThanWeek']]
    Y = dataset[['Failed']]
    
    return X, Y

In [ ]:
def printResult(grid_result):
        #Printing Results / Best Parameters.
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [ ]:
X, Y = studentSetup()

In [ ]:
model, batch_size, epochs, param_grid = gs1Spec()

In [ ]:
#####Processing
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestBatchSize = grid_result.best_params_.get('batch_size')
bestEpochs = grid_result.best_params_.get('epochs')
printResult(grid_result)

In [ ]:
model, optimizer, param_grid = gs2Spec(batch_size=bestBatchSize, epochs=bestEpochs)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
#From GS2, we get the best optimizer.  Save this for future reference.  Pray it is not SGD.
bestOptimizer = grid_result.best_params_.get('optimizer')
printResult(grid_result)

In [ ]:
model, learn_rate, momentum, param_grid = gs3Spec(batch_size=bestBatchSize, epochs=bestEpochs)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
#we get no useful info from gs3 except what would be best to use for sgd
bestSGDLearn = grid_result.best_params_.get('learn_rate')
bestSGDMomentum = grid_result.best_params_.get('momentum')

printResult(grid_result)

In [ ]:
model, init_mode, param_grid = gs4Spec(batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
#from gs4, we get the best init mode.
bestInitMode = grid_result.best_params_.get('init_mode')
printResult(grid_result)

In [11]:
model, activation, param_grid = gs5Spec(batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode)

NameError: name 'gs5Spec' is not defined

In [12]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

NameError: name 'model' is not defined

In [ ]:
bestActivation = grid_result.best_params_.get('activation')
printResult(grid_result)

In [ ]:
model, weight_constraint, dropout_rate, param_grid = gs6Spec(batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode, activation=bestActivation)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
#Pick these up for later print-off.
bestDropoutRate = grid_result.best_params_.get('dropout_rate')
bestWeightConstraint = grid_result.best_params_.get('weight_constraint')
printResult(grid_result)

In [ ]:
model, neurons, param_grid = gs7Spec(batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode, dropout_rate = bestDropoutRate, weight_constraint=bestWeightConstraint)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestNeurons = grid_result.best_params_.get('neurons')
printResult(grid_result)